In [21]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('database/database.sqlite')

# Crear un cursor para ejecutar consultas
cursor = conn.cursor()
# Obtener la lista de tablas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

# Mostrar las tablas
print("Tablas en la base de datos:")
for tabla in tablas:
    print(tabla[0])

Tablas en la base de datos:
apicall
commerce


In [22]:
# Obtener información sobre las columnas de la tabla apicall
cursor.execute(f"PRAGMA table_info({'apicall'});")
columnas = cursor.fetchall()

# Mostrar las columnas y sus tipos de datos
print(f"\nColumnas en la tabla '{'apicall'}':")
for columna in columnas:
    print(f"Nombre: {columna[1]}, Tipo: {columna[2]}")


Columnas en la tabla 'apicall':
Nombre: date_api_call, Tipo: TEXT
Nombre: commerce_id, Tipo: TEXT
Nombre: ask_status, Tipo: TEXT
Nombre: is_related, Tipo: REAL


In [16]:
# Obtener información sobre las columnas de la tabla commerce
cursor.execute(f"PRAGMA table_info({'commerce'});")
columnas = cursor.fetchall()

# Mostrar las columnas y sus tipos de datos
print(f"\nColumnas en la tabla '{'commerce'}':")
for columna in columnas:
    print(f"Nombre: {columna[1]}, Tipo: {columna[2]}")


Columnas en la tabla 'commerce':
Nombre: commerce_id, Tipo: TEXT
Nombre: commerce_nit, Tipo: INTEGER
Nombre: commerce_name, Tipo: TEXT
Nombre: commerce_status, Tipo: TEXT
Nombre: commerce_email, Tipo: TEXT


In [17]:
import pandas as pd

# Cargar los datos de una tabla en un DataFrame
query = f"SELECT * FROM {'apicall'};"
df_apicall = pd.read_sql_query(query, conn)

# Cargar los datos de una tabla en un DataFrame
query = f"SELECT * FROM {'commerce'};"
df_commerce = pd.read_sql_query(query, conn)


In [18]:
# Unir las tablas apicall y commerce
df = pd.merge(df_apicall, df_commerce, on='commerce_id')

# Filtrar solo las empresas activas
df = df[df['commerce_status'] == 'Active']

df['date_api_call'] = pd.to_datetime(df['date_api_call'])
#Cambiamos el tipo de datos a fecha ya que en un principio es  de tipo object

# Filtrar por julio y agosto de 2024
df = df[
    (df['date_api_call'].dt.month.isin([7, 8])) &
    (df['date_api_call'].dt.year == 2024)
]

In [31]:
def calcular_comision(empresa, peticiones_exitosas, peticiones_no_exitosas):
    comision = 0

    # Calculamos la comisión base
    if empresa == "Innovexa Solutions":
        comision = peticiones_exitosas * 300
    elif empresa == "NexaTech Industries":
        if 0<= peticiones_exitosas <= 10000:
            comision = peticiones_exitosas * 250
        elif 10001 <= peticiones_exitosas <= 20000:
            comision = peticiones_exitosas * 200
        else:
            comision = peticiones_exitosas * 170
    elif empresa == "QuantumLeap Inc.":
        comision = peticiones_exitosas * 600
    elif empresa == "Zenith Corp.":
        if 0<= peticiones_exitosas <= 22000:
            comision = peticiones_exitosas * 250
        else:
            comision = peticiones_exitosas * 130
    elif empresa == "FusionWave Enterprises":
        comision = peticiones_exitosas * 300

    # Calculamos el iva (antes de aplicar descuentos)
    iva = comision * 0.19

    # Aplicamos descuentos sobre la comisión
    if empresa == "Zenith Corp." and peticiones_no_exitosas > 6000:
        descuento = comision * 0.05
        comision -= descuento
    elif empresa == "FusionWave Enterprises":
        if 2500 <= peticiones_no_exitosas <= 4500:
            descuento = comision * 0.05
            comision -= descuento
        elif peticiones_no_exitosas > 4500:
            descuento = comision * 0.08
            comision -= descuento

    # Calculamos el valor total
    valor_total = comision + iva

    # Retornamos la comisión, el IVA y el valor total
    return comision, iva, valor_total


# Ejemplo 3: FusionWave Enterprises con descuento
comision, iva, valor_total = calcular_comision("FusionWave Enterprises", 12000, 5000)
print(f"FusionWave Enterprises - Comisión: {comision}, IVA: {iva}, Valor Total: {valor_total}")

FusionWave Enterprises - Comisión: 3312000.0, IVA: 684000.0, Valor Total: 3996000.0


In [35]:
def calcular_comision(empresa, peticiones_exitosas, peticiones_no_exitosas):
    comision = 0

    # Calculamos la comisión base según el contrato de la empresa
    if empresa == "Innovexa Solutions":
        comision = peticiones_exitosas * 300
    elif empresa == "NexaTech Industries":
        if peticiones_exitosas <= 10000:
            comision = peticiones_exitosas * 250
        elif 10001 <= peticiones_exitosas <= 20000:
            comision = (10000 * 250) + ((peticiones_exitosas - 10000) * 200)
        else:
            comision = (10000 * 250) + (10000 * 200) + ((peticiones_exitosas - 20000) * 170)
    elif empresa == "QuantumLeap Inc.":
        comision = peticiones_exitosas * 600
    elif empresa == "Zenith Corp.":
        if peticiones_exitosas <= 22000:
            comision = peticiones_exitosas * 250
        else:
            comision = (22000 * 250) + ((peticiones_exitosas - 22000) * 130)
    elif empresa == "FusionWave Enterprises":
        comision = peticiones_exitosas * 300

    # Calculamos el IVA (19% sobre la comisión)
    #iva = comision * 0.19

    # Aplicamos descuentos sobre la comisión (antes de IVA)
    if empresa == "Zenith Corp." and peticiones_no_exitosas > 6000:
        descuento = comision * 0.05
        comision -= descuento
    elif empresa == "FusionWave Enterprises":
        if 2500 <= peticiones_no_exitosas <= 4500:
            descuento = comision * 0.05
            comision -= descuento
        elif peticiones_no_exitosas > 4500:
            descuento = comision * 0.08
            comision -= descuento


    # Calculamos el valor total (comisión + IVA)
    valor_total = comision + iva

    # Retornamos la comisión, el IVA y el valor total
    return comision, iva, valor_total

# Ejemplo 3: FusionWave Enterprises con descuento
comision, iva, valor_total = calcular_comision("Zenith Corp.", 25000, 7000)
print(f"FusionWave Enterprises - Comisión: {comision}, IVA: {iva}, Valor Total: {valor_total}")

FusionWave Enterprises - Comisión: 5595500.0, IVA: 1063145.0, Valor Total: 6658645.0


In [37]:
def calcular_comision(empresa, peticiones_exitosas, peticiones_no_exitosas):
    comision = 0

    # Calculamos la comisión base según el contrato de la empresa
    if empresa == "Innovexa Solutions":
        comision = peticiones_exitosas * 300
    elif empresa == "NexaTech Industries":
        if peticiones_exitosas <= 10000:
            comision = peticiones_exitosas * 250
        elif 10001 <= peticiones_exitosas <= 20000:
            comision = (10000 * 250) + ((peticiones_exitosas - 10000) * 200)
        else:
            comision = (10000 * 250) + (10000 * 200) + ((peticiones_exitosas - 20000) * 170)
    elif empresa == "QuantumLeap Inc.":
        comision = peticiones_exitosas * 600
    elif empresa == "Zenith Corp.":
        if peticiones_exitosas <= 22000:
            comision = peticiones_exitosas * 250
        else:
            comision = (22000 * 250) + ((peticiones_exitosas - 22000) * 130)
    elif empresa == "FusionWave Enterprises":
        comision = peticiones_exitosas * 300

    # Aplicamos descuentos sobre la comisión (antes de calcular el IVA)
    if empresa == "Zenith Corp." and peticiones_no_exitosas > 6000:
        descuento = comision * 0.05
        comision -= descuento
    elif empresa == "FusionWave Enterprises":
        if 2500 <= peticiones_no_exitosas <= 4500:
            descuento = comision * 0.05
            comision -= descuento
        elif peticiones_no_exitosas > 4500:
            descuento = comision * 0.08
            comision -= descuento

    # Calculamos el IVA (19% sobre la comisión final)
    iva = comision * 0.19

    # Calculamos el valor total (comisión + IVA)
    valor_total = comision + iva

    # Retornamos la comisión, el IVA y el valor total
    return comision, iva, valor_total

# Ejemplo 3: FusionWave Enterprises con descuento
comision, iva, valor_total = calcular_comision("FusionWave Enterprises", 12000, 5000)
print(f"FusionWave Enterprises - Comisión: {comision}, IVA: {iva}, Valor Total: {valor_total}")


FusionWave Enterprises - Comisión: 3312000.0, IVA: 629280.0, Valor Total: 3941280.0


In [20]:
# Cerrar la conexión a la base de datos
conn.close()